In [1]:
from utils.read_datasetBreakfast import load_data, read_mapping_dict
import os
import numpy as np


COMP_PATH = ''

''' 
training to load train set
test to load test set
'''
split = 'training'
#split = 'test'
train_split =  os.path.join(COMP_PATH, 'splits/train.exclude_val.bundle') #Train Split
test_split  =  os.path.join(COMP_PATH, 'splits/test.split1.bundle') #Test Split
GT_folder   =  os.path.join(COMP_PATH, 'groundTruth/') #Ground Truth Labels for each training video 
DATA_folder =  os.path.join(COMP_PATH, 'Data/') #Frame I3D features for all videos
mapping_loc =  os.path.join(COMP_PATH, 'splits/mapping_bf.txt') 

actions_dict = read_mapping_dict(mapping_loc)

data_feat, data_labels = load_data( train_split, actions_dict, GT_folder, DATA_folder, datatype = split) #Get features and labels

Finish Load the Training data and labels!!!


In [2]:
validation_split = os.path.join(COMP_PATH, 'splits/val.split1.bundle') #Validation split
val_data_feat, val_data_labels = load_data( validation_split, actions_dict, GT_folder, DATA_folder, datatype = split) #Get features and labels

Finish Load the Training data and labels!!!


In [19]:
import torch
def splitByFrames(data_feat, data_labels, frames_per_clip):
    new_data_feat = []
    new_data_labels = []
    for i in range(0, len(data_feat)):
        total_frames, dim = data_feat[i].shape
        segment = data_feat[i]
        if total_frames % frames_per_clip != 0:
            pad_num = frames_per_clip - total_frames % frames_per_clip
            pad_Tensor = torch.zeros((pad_num, 400), dtype=torch.float64)
            segment = torch.cat((segment, pad_Tensor))
        total_frames, dim = segment.shape
        clip_num = int(total_frames / frames_per_clip)
        clips = segment.view(-1, frames_per_clip, 400)
    
        label = data_labels[i]
        new_data_labels += [label] * clip_num
        new_data_feat.append(clips)

    new_data_feat = torch.cat(new_data_feat)
    return new_data_feat, new_data_labels

new_data_feat, new_data_labels = splitByFrames(data_feat, data_labels, 16)
val_data_feat, val_data_labels = splitByFrames(val_data_feat, val_data_labels, 16)

In [21]:
import torch
from Dataset.VideoDataset import VideoDataset
import torch.utils.data as tud
from Models.LSTM import LSTM_Model
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

epochs = 10
batch_size = 50

dataset = VideoDataset(new_data_feat, new_data_labels)
dataloader = tud.DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = LSTM_Model()
learning_rate = 1e-3
log_interval = 200

def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    
    losses = []
    scores = []
    for batch_idx, (in_feature, label) in enumerate(train_loader):
        in_feature = in_feature.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()
        output = model(in_feature)
        loss = F.cross_entropy(output, label)
        losses.append(loss.item())
        
        label_predict = torch.max(output, 1)[1]
        step_score = accuracy_score(label.cpu().data.squeeze().numpy(), label_predict.cpu().data.squeeze().numpy())
        scores.append(step_score)
        
        loss.backward()
        optimizer.step()
        
        if (batch_idx + 1) % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accu: {:.2f}%'.format(
                epoch + 1, (batch_idx+1)*batch_size, len(train_loader.dataset), 100. * (batch_idx + 1) / len(train_loader), loss.item(), 100 * step_score))
        
    return losses, scores

In [23]:
val_dataset = VideoDataset(val_data_feat, val_data_labels)
val_dataloader = tud.DataLoader(val_dataset, batch_size= batch_size, shuffle=True)

def validation(model, device, test_loader):
    model.eval()
    
    test_loss = 0
    all_labels = []
    all_labels_predict = []
    with torch.no_grad():
        for in_feature, labels in test_loader:
            in_feature = in_feature.to(device)
            labels = labels.to(device)
            
            output = model(in_feature)
            loss = F.cross_entropy(output, labels)
            test_loss += loss.item()
            labels_predict = torch.max(output, 1)[1]
            all_labels.extend(labels)
            all_labels_predict.extend(labels_predict)
            
    test_loss = test_loss / len(test_loader.dataset)
    # compute accuracy
    all_labels = torch.stack(all_labels, dim=0)
    all_labels_predict = torch.stack(all_labels_predict, dim=0)
    test_score = accuracy_score(all_labels.cpu().data.squeeze().numpy(), all_labels_predict.cpu().data.squeeze().numpy())
    print('\nTest set ({:d} samples): Average loss: {:.4f}, Accuracy: {:.2f}%\n'.format(len(all_labels), test_loss, 100* test_score))
    
    return test_loss, test_score

In [24]:
cuda_avail = torch.cuda.is_available()
device = torch.device("cuda" if cuda_avail else "cpu")

lstm = LSTM_Model().double().to(device)
optimizer = torch.optim.Adam(list(lstm.parameters()), lr=learning_rate)

# record training process
epoch_train_losses = []
epoch_train_scores = []
epoch_test_losses = []
epoch_test_scores = []

for epoch in range(epochs):
    train_losses, train_scores = train(log_interval, lstm, device, dataloader, optimizer, epoch)
    test_losses, test_score = validation(lstm, device, val_dataloader)
    
    epoch_train_losses.append(train_losses)
    epoch_train_scores.append(train_scores)
    epoch_test_losses.append(test_losses)
    epoch_test_scores.append(test_score)

torch.save(model.state_dict(), "./trained/lstm.pt")
A = np.array(epoch_train_losses)
B = np.array(epoch_train_scores)
C = np.array(epoch_test_losses)
D = np.array(epoch_test_scores)
np.save('./results/lstm/training_losses_2.npy', A)
np.save('./results/lstm/training_scores_2.npy', B)
np.save('./results/lstm/test_loss.npy', C)
np.save('./results/lstm/test_score.npy', D)
    

Train Epoch: 1 [10000/146292 (7%)]	Loss: 3.091752, Accu: 14.00%
Train Epoch: 1 [20000/146292 (14%)]	Loss: 2.869417, Accu: 20.00%
Train Epoch: 1 [30000/146292 (21%)]	Loss: 2.814002, Accu: 22.00%
Train Epoch: 1 [40000/146292 (27%)]	Loss: 2.535698, Accu: 24.00%
Train Epoch: 1 [50000/146292 (34%)]	Loss: 2.557809, Accu: 24.00%
Train Epoch: 1 [60000/146292 (41%)]	Loss: 2.352424, Accu: 20.00%
Train Epoch: 1 [70000/146292 (48%)]	Loss: 1.964300, Accu: 44.00%
Train Epoch: 1 [80000/146292 (55%)]	Loss: 2.134269, Accu: 38.00%
Train Epoch: 1 [90000/146292 (62%)]	Loss: 2.048891, Accu: 36.00%
Train Epoch: 1 [100000/146292 (68%)]	Loss: 2.216703, Accu: 38.00%
Train Epoch: 1 [110000/146292 (75%)]	Loss: 2.221876, Accu: 32.00%
Train Epoch: 1 [120000/146292 (82%)]	Loss: 2.377602, Accu: 34.00%
Train Epoch: 1 [130000/146292 (89%)]	Loss: 1.912910, Accu: 38.00%
Train Epoch: 1 [140000/146292 (96%)]	Loss: 1.938234, Accu: 42.00%

Test set (34543 samples): Average loss: 0.0489, Accuracy: 32.19%

Train Epoch: 2 [100

KeyboardInterrupt: 

In [35]:
import datetime
print(datetime.datetime.now().date())

2020-03-28


In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(10, 4))
plt.subplot(121)
#plt.plot(np.arange(1, epochs + 1), A[:, -1])  # train loss (on epoch end)
plt.plot(A.flatten())  # train loss (on epoch end)
#plt.plot(np.arange(1, epochs + 1), C)  
plt.title("model loss")
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(['train','test'], loc="upper left")
# 2nd figure
plt.subplot(122)
#plt.plot(np.arange(1, epochs + 1), B[:, -1])
plt.plot(B.flatten())

#plt.plot(np.arange(1, epochs + 1), D) # train accuracy (on epoch end)
plt.title("training scores")
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(['train', 'test'], loc="upper left")
plt.show()

